In [ ]:
OFFLINE = True

In [1]:
import os, json, time
import gc
from IPython.display import display, Markdown

In [5]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import transformers
import torch
from transformers import AutoTokenizer,BitsAndBytesConfig,AutoModelForCausalLM, TrainingArguments
from langchain_community.document_loaders import TextLoader
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import ConfigurableField
from langchain_community.vectorstores import FAISS, Chroma
# Text embedding / Texxt Splitter for RAG 
from langchain_text_splitters import RecursiveCharacterTextSplitter, CharacterTextSplitter, SentenceTransformersTokenTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from datasets import Dataset, DatasetDict, load_dataset

# Adv RAG library 

# for evaluate  LLM 
import evaluate # require online 
# from deepeval.metrics import GEval
# from deepeval.test_case import LLMTestCase
# from deepeval.metrics import AnswerRelevancyMetric, ContextualPrecisionMetric , ContextualRelevancyMetric, ContextualRecallMetric
import pytest
# import trulens

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device


device(type='cpu')

In [7]:
def clearMemory():
    for _ in range(5):
        torch.cuda.empty_cache()
        gc.collect()
        time.sleep(0.3)

In [ ]:
# Enable/Disable Function
FEW_SHOT_TEST= False#True
USE_RAG = True#False#False #True#True
USE_WANDB = False#True # for  LLM evalution and debug , track fine tuning performance
USE_TRULENS = False # for LLM evalution
USE_DEEPEVAL = False # for LLM evalution   (require openAI API key)
USE_TRAIN =  True #True #False#True 
USE_INFER =  False # for submision prediction only , no test model 
if OFFLINE :
    USE_WANDB = False # Wandb only support online  
if device.type == "cpu": #requred GPU support for fine turning 
    USE_TRAIN= False

if USE_WANDB:
    # train report to  W&B tool
    import wandb
    from kaggle_secrets import UserSecretsClient
    reportTo= "wandb"
    user_secrets = UserSecretsClient()
    my_secret = user_secrets.get_secret("wandb_api_key") 
    wandb.login(key=my_secret) # login 
else: 
    reportTo = "none"# None
#     os.environ["WANDB_DISABLED"] = True#“true”
    